In [1]:
import nltk
import random
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity      
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
# import spacy
lemmatizer = nltk.stem.WordNetLemmatizer()

# Download required NLTK data
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\flora\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\flora\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\flora\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
data = pd.read_csv('Samsung Dialog.txt', sep = ':', header = None)
data

,0,1
0,Customer,"Hi, I'm looking to buy a new phone, and I'm i..."
1,Sales Agent,"Great, we have a wide range of Samsung phones..."
2,Customer,"Well, I want a phone with a good camera, long..."
3,Sales Agent,Absolutely. We have a lot of great options th...
4,Customer,"No, I haven't. Tell me more about it."
5,Sales Agent,The Galaxy S21 Ultra has a 108-megapixel came...
6,Customer,That sounds great. How much does it cost?
7,Sales Agent,"The Galaxy S21 Ultra starts at $1,199, but we..."
8,Customer,"Okay, I'm interested. But I have a few more q..."
9,Sales Agent,The Galaxy S21 Ultra comes with a standard on...


In [3]:
cust = data.loc[data[0] == 'Customer']
sales = data.loc[data[0] == 'Sales Agent']

sales = sales[1].reset_index(drop = True)
cust = cust[1].reset_index(drop = True)

new_data = pd.DataFrame()
new_data['Question'] = cust
new_data['Answer'] = sales

new_data.head()


,Question,Answer
0,"Hi, I'm looking to buy a new phone, and I'm i...","Great, we have a wide range of Samsung phones..."
1,"Well, I want a phone with a good camera, long...",Absolutely. We have a lot of great options th...
2,"No, I haven't. Tell me more about it.",The Galaxy S21 Ultra has a 108-megapixel came...
3,That sounds great. How much does it cost?,"The Galaxy S21 Ultra starts at $1,199, but we..."
4,"Okay, I'm interested. But I have a few more q...",The Galaxy S21 Ultra comes with a standard on...


In [4]:
# Define a function for text preprocessing (including lemmatization)
def preprocess_text(text):
    # Identifies all sentences in the data
    sentences = nltk.sent_tokenize(text)
    
    # Tokenize and lemmatize each word in each sentence
    preprocessed_sentences = []
    for sentence in sentences:
        tokens = [lemmatizer.lemmatize(word.lower()) for word in nltk.word_tokenize(sentence) if word.isalnum()]
        # Turns to basic root - each word in the tokenized word found in the tokenized sentence - if they are all alphanumeric 
        # The code above does the following:
        # Identifies every word in the sentence 
        # Turns it to a lower case 
        # Lemmatizes it if the word is alphanumeric

        preprocessed_sentence = ' '.join(tokens)
        preprocessed_sentences.append(preprocessed_sentence)
    
    return ' '.join(preprocessed_sentences)


new_data['tokenized Questions'] = new_data['Question'].apply(preprocess_text)
new_data

,Question,Answer,tokenized Questions
0,"Hi, I'm looking to buy a new phone, and I'm i...","Great, we have a wide range of Samsung phones...",hi i looking to buy a new phone and i interest...
1,"Well, I want a phone with a good camera, long...",Absolutely. We have a lot of great options th...,well i want a phone with a good camera long ba...
2,"No, I haven't. Tell me more about it.",The Galaxy S21 Ultra has a 108-megapixel came...,no i have tell me more about it
3,That sounds great. How much does it cost?,"The Galaxy S21 Ultra starts at $1,199, but we...",that sound great how much doe it cost
4,"Okay, I'm interested. But I have a few more q...",The Galaxy S21 Ultra comes with a standard on...,okay i interested but i have a few more questi...
5,That's good to know. And what about the opera...,"Yes, the Galaxy S21 Ultra runs on Android 11,...",that good to know and what about the operating...
6,"Okay, that's good. But I'm also interested in...",Absolutely. The Galaxy A52 is a great mid-ran...,okay that good but i also interested in some o...
7,That sounds like a good option for me. How mu...,"The Galaxy A52 starts at $399, but again, we ...",that sound like a good option for me how much ...
8,"Okay, I'll think about it. But can you also t...",Of course. The Galaxy Z Fold2 is a really uni...,okay i think about it but can you also tell me...
9,"That sounds really cool, but it also sounds e...","The Galaxy Z Fold2 starts at $1,999, but agai...",that sound really cool but it also sound expen...


In [5]:
xtrain = new_data['tokenized Questions'].to_list()
xtrain 

['hi i looking to buy a new phone and i interested in samsung phone',
 'well i want a phone with a good camera long battery life and plenty of storage',
 'no i have tell me more about it',
 'that sound great how much doe it cost',
 'okay i interested but i have a few more question what kind of warranty come with the phone',
 'that good to know and what about the operating system doe it come with the latest version of android',
 'okay that good but i also interested in some of the other samsung phone can you tell me more about the galaxy a52',
 'that sound like a good option for me how much doe it cost',
 'okay i think about it but can you also tell me about the galaxy z fold2 i heard a lot about it and i curious',
 'that sound really cool but it also sound expensive how much doe it cost',
 'hmm i not sure that a lot of money for a phone',
 'okay can you tell me more about the galaxy a72',
 'that sound like a great option for me how much doe it cost',
 'okay i definitely consider it but

In [6]:
# Vectorize corpus
tfidf_vectorizer = TfidfVectorizer()
corpus = tfidf_vectorizer.fit_transform(xtrain)

print(corpus)

  (0, 71)	0.3011232459093611
  (0, 41)	0.2700934382194581
  (0, 42)	0.2460248494975018
  (0, 5)	0.22635938061087743
  (0, 67)	0.4194649271573511
  (0, 58)	0.34485730351794186
  (0, 11)	0.34485730351794186
  (0, 84)	0.2700934382194581
  (0, 51)	0.34485730351794186
  (0, 37)	0.34485730351794186
  (1, 75)	0.3135527535104317
  (1, 61)	0.2058115236343365
  (1, 68)	0.3135527535104317
  (1, 48)	0.3135527535104317
  (1, 8)	0.3135527535104317
  (1, 50)	0.3135527535104317
  (1, 13)	0.3135527535104317
  (1, 32)	0.2058115236343365
  (1, 90)	0.22369185226757943
  (1, 86)	0.3135527535104317
  (1, 88)	0.3135527535104317
  (1, 5)	0.2058115236343365
  (1, 67)	0.19069392118064465
  (2, 44)	0.2707901771636704
  (2, 2)	0.30884634876971573
  :	:
  (15, 31)	0.33865461688136433
  (15, 22)	0.2957071705810691
  (15, 1)	0.2957071705810691
  (15, 83)	0.2957071705810691
  (15, 30)	0.2415997872161552
  (15, 82)	0.17934057989570892
  (15, 63)	0.19181632811123442
  (15, 39)	0.22228802619600418
  (15, 33)	0.265235472

In [7]:
user = input('Pls ask your question')
print(user)

i need a phone


In [8]:
preprocess_text(user)

'i need a phone'

In [9]:
# vectorize user input 
user_transformed = tfidf_vectorizer.transform([user])
print(user_transformed)

  (0, 67)	1.0


In [10]:
# find similarity 
similarity_scores = cosine_similarity(user_transformed, corpus)
similarity_scores

array([[0.41946493, 0.19069392, 0.        , 0.        , 0.19974086,
        0.        , 0.17724721, 0.        , 0.        , 0.        ,
        0.24249352, 0.        , 0.        , 0.        , 0.18308115,
        0.        , 0.        ]])

In [11]:
new_data['Answer'].iloc[similarity_scores.argmax()]

' Great, we have a wide range of Samsung phones to choose from. What features are you looking for in a phone?'

In [12]:
user = input('Pls ask your question')
print(user)

i need a phone 


In [13]:
preprocess_text(user)

'i need a phone'

In [14]:
# vectorize user input 
user_transformed = tfidf_vectorizer.transform([user])
print(user_transformed)

  (0, 67)	1.0


In [15]:
# find similarity 
similarity_scores = cosine_similarity(user_transformed, corpus)
similarity_scores

array([[0.41946493, 0.19069392, 0.        , 0.        , 0.19974086,
        0.        , 0.17724721, 0.        , 0.        , 0.        ,
        0.24249352, 0.        , 0.        , 0.        , 0.18308115,
        0.        , 0.        ]])

In [16]:
new_data['Answer'].iloc[similarity_scores.argmax()]

' Great, we have a wide range of Samsung phones to choose from. What features are you looking for in a phone?'

In [17]:
def collector():
    user = input('Pls ask your question: ')
    pre_user = preprocess_text(user)
    vect_user = tfidf_vectorizer.transform([pre_user])
    similarity_scores = cosine_similarity(vect_user, corpus)
    most_similar_index = similarity_scores.argmax()
    
    return new_data['Answer'].iloc[most_similar_index]


In [18]:
collector()

" You're welcome. If you have any other questions or concerns, don't hesitate to reach out to us. We're always here to help."

In [19]:
def responder(text):
    user_input_processed = preprocess_text(user_input)
    vectorized_user_input = tfidf_vectorizer.transform([user_input_processed])
    similarity_scores = cosine_similarity(vectorized_user_input, corpus)
    argument_maximum = similarity_scores.argmax()
    return new_data['Answer'].iloc[argument_maximum]

bot_greetings = ['Hello user, You are chatting with Flora....How may i help you',
             'Hi Dear, i am here if you need me',
             'Hey, what do need me to do',
             'Hello i am here for you',
             'How can i be of help' ]

bot_farewell = ['Thanks for your usage....bye',
            'Alright dear...Hope to see you soon',
            'Hope i was of help to know...Bye',
            'Do you have anymore question you to ask....Bye',
            'Thanks for reaching out...Hope i answer all your question...Bye']

human_greeting = ['Hi', 'Hello There', 'Hiyya', 'Hey', 'hello', 'Wassup']

human_exits = ['Thanks bye', 'bye', 'quite', 'exit', 'bye bye', 'close']

import random
random_greeting = random.choice(bot_greetings)
random_farewell = random.choice(bot_farewell)

while True:
    user_input = input('You: ')

    if user_input.lower() in human_greeting:
        print(random_greeting)
    elif user_input.lower() in human_exits:
        print(random_farewell)
        break
    else:
      responder(user_input)



KeyboardInterrupt: Interrupted by user

In [ ]:
def get_response(user_input):
    user_input_processed = preprocess_text(user_input) # ....................... Preprocess the user's input using the preprocess_text function

    user_input_vector = tfidf_vectorizer.transform([user_input_processed])# .... Vectorize the preprocessed user input using the TF-IDF vectorizer

    similarity_scores = cosine_similarity(user_input_vector, corpus) # .. Calculate the score of similarity between the user input vector and the corpus (df) vector

    most_similar_index = similarity_scores.argmax() # ..... Find the index of the most similar question in the corpus (df) based on cosine similarity

    return new_data['Answer'].iloc[most_similar_index] # ... Retrieve the corresponding answer from the df DataFrame and return it as the chatbot's response

# create greeting list 
greetings = ["Hey There.... I am a creation of Ehiz Danny Agba Coder.... How can I help",
            "Hi Human.... How can I help",
            'Twale baba nla, wetin dey happen nah',
            'How far Alaye, wetin happen'
            "Good Day .... How can I help", 
            "Hello There... How can I be useful to you today",
            "Hi GomyCode Student.... How can I be of use"]

exits = ['thanks bye', 'bye', 'quit', 'exit', 'bye bye', 'close']
farewell = ['Thanks....see you soon', 'Babye, See you soon', 'Bye... See you later', 'Bye... come back soon']

random_farewell = random.choice(farewell) # ---------------- Randomly select a farewell message from the list
random_greetings = random.choice(greetings) # -------- Randomly select greeting message from the list

# Test your chatbot
while True:
    user_input = input("You: ")
    if user_input.lower() in exits:
        print(f"\nChatbot: {random_farewell}!")
        break
    if user_input.lower() in ['hi', 'hello', 'hey', 'hi there']:
        print(f"\nChatbot: {random_greetings}!")
    else:   
        response = get_response(user_input)
        print(f"\nChatbot: {response}")